<a href="https://colab.research.google.com/github/Baco23/github-course/blob/master/LSTM_Lotto_Quina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

Prepare/Generate data set //
let´s imagine that we have history of lottery by generating 80 numbers


In [3]:
df = pd.DataFrame(np.random.randint(1,80,size=(80, 5)), columns=list('ABCDE'))

The top games look like this (A B C D E F are balls that take range 1 to 60 each):

In [4]:
df.head()

,A,B,C,D,E
0,25,26,29,29,42
1,79,58,67,74,49
2,42,21,31,43,34
3,15,15,7,65,32
4,16,31,32,49,64


Next we need to normalize data.

In [6]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)

Lets define hyper params of or model

In [11]:
number_of_rows = df.values.shape[0] #all our games
window_length = 68 #amount of past games we need to take in consideration for prediction
number_of_features = df.values.shape[1]

Create train dataset and labels for each row. It should have format for keras lstm model(row, window sixe, balls)

In [12]:
train = np.empty([number_of_rows-window_length, window_length, number_of_features], dtype=float)

label = np.empty([number_of_rows-window_length, number_of_features], dtype=float)

window_length = 68

for i in range(0, number_of_rows-window_length):
  train[i]=transformed_df.iloc[i:i+window_length, 0: number_of_features]
  label[i]=transformed_df.iloc[i+window_length: i+window_length+1, 0:number_of_features]

Shapes

In [13]:
train.shape

(12, 68, 5)

In [14]:
label.shape

(12, 5)

In [15]:
train[0]

array([[-6.86517192e-01, -5.77890116e-01, -4.52508784e-01,
        -9.18157257e-01,  1.59768261e-03],
       [ 1.77468967e+00,  7.80602719e-01,  1.24161277e+00,
         1.38202794e+00,  2.99831770e-01],
       [ 8.83071907e-02, -7.90154622e-01, -3.63344492e-01,
        -2.02544085e-01, -3.39241275e-01],
       [-1.14229624e+00, -1.04487203e+00, -1.43331600e+00,
         9.21990899e-01, -4.24451014e-01],
       [-1.09671834e+00, -3.65625611e-01, -3.18762346e-01,
         1.04147274e-01,  9.38904815e-01],
       [-3.21893953e-01,  1.01356301e-01,  1.50910565e+00,
         1.58648884e+00, -7.22685102e-01],
       [-3.21893953e-01, -1.12977783e+00,  1.15244848e+00,
        -8.67042030e-01,  3.42436640e-01],
       [ 1.13659900e+00, -1.10908204e-01,  1.41994136e+00,
        -1.27596384e+00,  1.19453403e+00],
       [ 1.79463000e-01,  1.86262104e-01,  4.39134140e-01,
         9.21990899e-01,  5.12856119e-01],
       [ 2.70618810e-01, -1.59675974e+00, -1.29956956e+00,
        -1.83823133e+00

In [16]:
train[1]

array([[ 1.77468967e+00,  7.80602719e-01,  1.24161277e+00,
         1.38202794e+00,  2.99831770e-01],
       [ 8.83071907e-02, -7.90154622e-01, -3.63344492e-01,
        -2.02544085e-01, -3.39241275e-01],
       [-1.14229624e+00, -1.04487203e+00, -1.43331600e+00,
         9.21990899e-01, -4.24451014e-01],
       [-1.09671834e+00, -3.65625611e-01, -3.18762346e-01,
         1.04147274e-01,  9.38904815e-01],
       [-3.21893953e-01,  1.01356301e-01,  1.50910565e+00,
         1.58648884e+00, -7.22685102e-01],
       [-3.21893953e-01, -1.12977783e+00,  1.15244848e+00,
        -8.67042030e-01,  3.42436640e-01],
       [ 1.13659900e+00, -1.10908204e-01,  1.41994136e+00,
        -1.27596384e+00,  1.19453403e+00],
       [ 1.79463000e-01,  1.86262104e-01,  4.39134140e-01,
         9.21990899e-01,  5.12856119e-01],
       [ 2.70618810e-01, -1.59675974e+00, -1.29956956e+00,
        -1.83823133e+00, -1.70259710e+00],
       [-1.14229624e+00,  1.62966074e+00, -1.43331600e+00,
        -6.11465898e-01

In [17]:
label[0]

array([-0.45862767,  1.24758463, -0.72000166, -0.40700499, -1.7025971 ])

In [18]:
label[1]

array([ 0.04272929, -0.53543722, -0.54167308,  0.05303205,  0.08680742])

Our LSTM model

In [19]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import numpy as np

batch_size = 20

Using TensorFlow backend.


Using TensorFlow backend.

In [ ]:
model = Sequential()
model.add(LSTM(32,
               input_shape=(window_length, number_of_features),
               return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32,
               return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(number_of_features))
model.compile(loss='mse', optimizer='rmsprop')

Training

In [ ]:
model.fit(train, label,
          batch_size=64, epochs=3000)

Streaming output truncated to the last 5000 lines.
24/24 [==============================] - 0s 1ms/step - loss: 0.2558
Epoch 502/3000
24/24 [==============================] - 0s 1ms/step - loss: 0.2612
Epoch 503/3000
24/24 [==============================] - 0s 1ms/step - loss: 0.2492
Epoch 504/3000
24/24 [==============================] - 0s 1ms/step - loss: 0.2327
Epoch 505/3000
24/24 [==============================] - 0s 1ms/step - loss: 0.1991
Epoch 506/3000
24/24 [==============================] - 0s 1ms/step - loss: 0.2514
Epoch 507/3000
24/24 [==============================] - 0s 1ms/step - loss: 0.2153
Epoch 508/3000
24/24 [==============================] - 0s 1ms/step - loss: 0.2554
Epoch 509/3000
24/24 [==============================] - 0s 2ms/step - loss: 0.2348
Epoch 510/3000
24/24 [==============================] - 0s 1ms/step - loss: 0.2523
Epoch 511/3000
24/24 [==============================] - 0s 1ms/step - loss: 0.2391
Epoch 512/3000
24/24 [=============================

Prediction, lets imagene we know last 36 games and want to predict next game.

In [ ]:
to_predict=np.array([[47,29,42,4,27,21],
[14,55,35,30,18,57],
[7,53,43,20,45,38],
[60,27,28,20,53,11],
[57,36,11,58,55,45],
[27,39,7,31,46,45],
[25,9,7,10,58,19],
[56,18,26,13,1,40],
[24,42,18,5,25,9],
[14,38,28,18,35,54],
[11,5,42,3,34,37],
[33,11,14,34,15,18],
[9,33,46,24,27,5],
[42,53,47,1,44,48],
[20,9,42,29,30,15],
[4,49,9,31,47,53],
[39,52,27,57,58,33],
[58,22,26,15,54,5],
[46,50,1,17,37,30],
[32,39,55,33,52,31],
[47,4,24,44,56,9],
[20,15,57,49,39,41],
[9,37,30,10,54,47],
[18,30,21,31,34,51],
[26,7,1,23,14,5],
[54,56,58,20,27,41],
[37,12,14,34,35,47],
[23,26,51,27,29,7],
[7,30,46,23,8,14],
[43,49,57,33,9,24],
[29,19,8,2,37,3],
[50,20,46,23,14,39],
[23,37,59,10,58,31],
[48,49,53,20,32,33],
[4,30,52,23,13,28],
[55,14,23,29,11,1]])
scaled_to_predict = scaler.transform(to_predict) 

In [ ]:
scaled_predicted_output_1 = model.predict(np.array([scaled_to_predict]))
print(scaler.inverse_transform(scaled_predicted_output_1).astype(int)[0])

[27 22 39 25 42 50]
